# Object Detection กับ img

---

## ใช้ MobileNet_SSD กับรูป

In [1]:
import numpy as np
import cv2 

# --- Labels of Network. ---
# ได้จากตอนโหลด model zoo มา คือ list ของ obj. ที่ model นี้สามารถ detect ได้ 
# โดย model ที่อาจารย์ยกตัวอย่างมาคือ mobilenet_SSD และ VGG_SSD มีการใช้ classNames ร่วมกัน จึงมีการ detect obj. ได้เท่ากัน
classNames = { 0: 'background', # คือ ไม่เจออะไรเลย
    1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat',
    5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat', 9: 'chair',
    10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse',
    14: 'motorbike', 15: 'person', 16: 'pottedplant',
    17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor' }


# --- Load the Caffe model ---
# สร้างมาจากแพลตฟอร์ม Caffe
# parameter ตัวแรก: MobileNetSSD_deploy.prototxt คือ จะเป็นตัวบอกข้อมูลข้างในของ network ว่ามีอะไรบ้าง
# parameter ตัวที่สอง: MobileNetSSD_deploy.caffemodel คือ weight ของ model ว่า train มาอย่างไร
net = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt', 'MobileNetSSD_deploy.caffemodel') 

# --- เตรียมรูป ---
# โหลดรูปที่จะใช้
img = cv2.imread('ways.jpg')

# convert รูปเป็น blob file
blob = cv2.dnn.blobFromImage(img, 0.007843, (300, 300), (127.5, 127.5, 127.5), False)

# Set to network the input blob เอา blob มาใส่เป็น input
net.setInput(blob)

# Prediction of network 
# สั่งให้คำนวน จะได้เป็น list obj. ที่หาเจอ
detections = net.forward()

# Size of image 
# เก็บค่าของ height และ width ของรูปเดิมไว้ก่อน เพื่อจะได้คูณกลับเป็นรูปเดิมตอนถูกย่อขนาดรูปแล้ว
height = img.shape[0]  
width = img.shape[1] 

# --- Locate location and class of object detected ---
# There is a fix index for class, location and confidence
# loop เข้าไปใน list ของ detections ที่เกิดขึ้น ว่าในนั้นมีกี่ตัว และแต่ละตัวเป็นตำแหน่งไหนบ้าง
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2] #Confidence of prediction 
    # ตัว detections จะมีรูปแบบมิติของ matrix อยู่ประมาณนี้ [0, 0, i, 2] โดย index ที่ 1, 2 จะเป็น dimention ว่าง หรือ 1 
    # ซึ่งเราสนใจที่ index 3, 4 
        # โดย index ที่ 3 คือ ลำดับของ obj. ที่เราเจอว่าเป็นตัวที่เท่าไร เช่น ตัวที่ 1 ในรูป ตัวที่ 2 ในรูป 
        # index ที่ 4 ช่องที่ 2 คือ ค่า confidence เป็นค่าความมั่นใจว่า obj. ที่เจอเป็นชนิดนีกี่ % 

    if confidence > 0.3: # Filter prediction 
        class_id = int(detections[0, 0, i, 1]) # Class label # index ที่ 4 ช่องที่ 1 คือ class_id ว่าคือชนิดใด
        
        # Scale detection frame ที่จะเอามาวาดกรอบบนรูป
        # โดยเอาค่าขนาดรูปเดิมมาคูณกับค่าที่ได้ เพราะค่าที่ได้ถูก normalize จนเหลือ 0 กับ 1 เท่านั้น พอเอามาคุณกับค่าเดิมของมันก็จะได้ขนาดรูปเดิมมาที่ตำแหน่งเดิม
        xLeftBottom = int(width * detections[0, 0, i, 3])  # index ที่ 4 ช่องที่ 3 คือ จุด xLeftBottom
        yLeftBottom = int(height * detections[0, 0, i, 4]) # index ที่ 4 ช่องที่ 4 คือ จุด yLeftBottom
        xRightTop   = int(width * detections[0, 0, i, 5]) # index ที่ 4 ช่องที่ 5 คือ จุด xRightTop
        yRightTop   = int(height * detections[0, 0, i, 6]) # index ที่ 4 ช่องที่ 6 คือ จุด yRightTop

        # Draw location เอามาวาดกรอบสี่เหลี่ยมทับตำแหน่งนั้นเลย เพื่อดูว่ามัน detect เจอจริง ๆ มั้ย
        cv2.rectangle(img, (xLeftBottom, yLeftBottom), (xRightTop, yRightTop),(0, 255, 0))

        # Draw label and confidence
        label = classNames[class_id] + ": " + str(confidence) # เอาค่า class_id (ชื่อชนิด)กับค่า confidence
        # พิมพ์ตัวอักษรใส่เข้าไปในรูป
        labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

        yLeftBottom = max(yLeftBottom, labelSize[1])
        # วาดสี่เหลี่ยมทับ และไฮไลท์พื้นหลังของตัวอักษรเป็นสีขาวจะได้เห็นชัด ๆ 
        cv2.rectangle(img, (xLeftBottom, yLeftBottom - labelSize[1]),
                                (xLeftBottom + labelSize[0], yLeftBottom + baseLine),
                                (255, 255, 255), cv2.FILLED)
        #วาด class_id (ชื่อชนิด)กับค่า confidence
        cv2.putText(img, label, (xLeftBottom, yLeftBottom),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

# แสดงรูป
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()



เมื่อ run เสร็จแล้วจะได้หน้าต่างด้านล่างขึ้นมา โดยจะแสดง obj. และค่า confidence นั้น ๆ ออกมา

![alt text](result_1.jpg)

จะเห็นว่า model นี้จะแม่นยำกับการตรวจจับรถมาก แต่ยังมีการตรวจจับ obj บางชนิดผิดอยู่ เช่น ตรวจจับเสาเป็นคน 
ส่วนตัวคิดว่ารูปนี้ถ้ามองไกล ๆ มองผ่าน ๆ เสาอันนี้ก็ดูเหมือนคนนิดนึง จึงไม่แปลกที่ model ก็อาจตรวจจับได้ เพราะข้อมูลที่ดีคือข้อมูลที่มนุษย์เราก็สามารถมองออกและมองรู้เรื่องว่ามันคืออะไรด้วย

---